# 🧑‍🎓Student:
## Marwan Mashra

In [60]:
import random
from tabulate import tabulate
from collections import Counter
from tqdm import tqdm
import math


In [61]:
class CountMinSketch:
    def __init__(self, d, w, universe=['A','B','C','D','E']):
        """_summary_

        Args:
            d (int): number of hash functions
            w (int): size of hash 
        """
        self.d = d 
        self.w = w
        self.universe = universe

        self.p = 1223543677 # a (relatively) large prime
        self.epsilon = math.e/self.w
        self.delta = 1/math.exp(self.d)

        self.a = [random.randrange(self.p) for i in range(d)]
        self.b = [random.randrange(self.p) for i in range(d)]

        self.array_init()

    def array_init(self):
        """ initialise all the array values to 0
        """
        self.array = [[0 for x in range(self.w)] for y in range(self.d)] 

    def hash(self, x, a, b):
        """ returns the hash of x in a hash function using a & b

        Args:
            x (any): element to be hashed
            a (float): values that will parametrize the hash function
            b (float): values that will parametrize the hash function

        Returns:
            int : the hash of x for the hash_a_b (integer between 0 and w-1)
        """
        return ((a*hash(x)+b)%self.p)%self.w

    def in_universe(self, x):
        """check if x is an element of the universe

        Args:
            x (any): element to check

        Returns:
            Bool: whether x is an element of the universe
        """
        if x in self.universe:
            return True
        else:
            print(f"ERROR: {x} is not in the universe")
            return False

    def get_hashes(self, x):
        """ computes all the hashes of x for the array a & b

        Args:
            x (any): element to be hashed

        Returns:
            list(int): a list of all hashes of x for the a,b array
        """
        hashes = []
        for a, b in zip(self.a, self.b):
            hashes.append(self.hash(x, a, b))
        return hashes

    def __str__(self):
        """ returns the table in which hashes are stored

        Returns:
            str: a string representing the table
        """
        header = [[f'h{d}']+self.array[d] for d in range(self.d)]
        return tabulate(header,tablefmt='grid')

    def update(self, x, show=False):
        """ update the table to add the element x to the count 

        Args:
            x (any): element of the universe to be added
            show (bool, optional): print the updated table. Defaults to False.
        """
        if self.in_universe(x):
            hashes = self.get_hashes(x)
            for i, h in enumerate(hashes):
                self.array[i][h] += 1

            if show:
                print(self)

    def query(self, x):
        """ returns the count of x

        Args:
            x (any): element of the universe for which we want the count

        Returns:
            int: count of x (>0)
        """
        if self.in_universe(x):
            hashes = self.get_hashes(x)
            counts = [self.array[i][h] for i, h in enumerate(hashes)]
            return min(counts)

        
    def random_init(self, k):
        """ generates k random element of the universe and add them to the table

        Args:
            k (int): how many element to add (>0)

        Returns:
            Counter: a counter object of all elements of the universe (see https://docs.python.org/3/library/collections.html#collections.Counter) 
        """
        self.array_init()
        choices = random.choices(self.universe, k = k)
        for x in tqdm(choices):
            self.update(x)
        return Counter(choices)

    def evaluate(self, test_size=100000):
        """ computes the loss for the given (ϵ,δ) approximation. 
        The loss is the sum of differences between the count and the approximated count divided by the sum of all the counts.

        Args:
            test_size (int, optional): the size of test for the evaluation, bigger = more precise. Defaults to 100000.
        """
        self.array_init()
        gold_counter = self.random_init(test_size)
        mse = []
        for x in self.universe:
            count = self.query(x)
            mse.append(abs(gold_counter[x]-count))

        loss = sum(mse)/test_size
        print(f'for (ϵ={self.epsilon}, δ={self.delta}) approximation, the loss is {loss}')
        
        self.array_init()
        

## Initialize the counter

In [62]:
counter = CountMinSketch(10, 30)  # 10 hash functions with arrays of size 30

print("Before updating")
for x in ['A','B','C']:
    print(f'{x} : {counter.query(x)}')

for x in ['A','A','A','B','B','C']:
    counter.update(x)

print("After updating")
for x in ['A','B','C']:
    print(f'{x} : {counter.query(x)}')


Before updating
A : 0
B : 0
C : 0
After updating
A : 3
B : 2
C : 1


## Print the table

In [63]:
print(counter)

+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| h0 | 0 | 0 | 0 | 0 | 0 | 3 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| h1 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| h2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 3 | 0 | 0 | 0 | 0 | 0 | 0 |
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| h3 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 3 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 

## Insert random elements

In [64]:
counter.random_init(100000)

100%|██████████| 100000/100000 [00:00<00:00, 154739.45it/s]


Counter({'C': 19702, 'D': 19916, 'B': 20151, 'A': 20073, 'E': 20158})

In [65]:
for x in counter.universe:
    print(f'{x} : {counter.query(x)}')

A : 20073
B : 20151
C : 19702
D : 19916
E : 20158


## Evaluate

The evaluation is done using the following Loss function:

$$

Loss = \frac{\sum_{i=0}^{len(U)-1} abs(RealCount(U_i) - CountMinSketch(U_i))}{\sum_{i=0}^{len(U)-1} RealCount(U_i)}

$$

$U$ : The Universe

$RealCount$ : The actual count

$CountMinSketch$ : The count using MinSketch

In [66]:
counter.evaluate()

100%|██████████| 100000/100000 [00:00<00:00, 152109.62it/s]

for (ϵ=0.09060939428196817, δ=4.539992976248485e-05) approximation, the loss is 0.0


### Evaluate different values for d & w

In [67]:
counter1 = CountMinSketch(5, 10)
counter1.evaluate()

100%|██████████| 100000/100000 [00:00<00:00, 264527.06it/s]

for (ϵ=0.2718281828459045, δ=0.006737946999085467) approximation, the loss is 0.0


In [68]:
counter2 = CountMinSketch(5, 5)
counter2.evaluate()

100%|██████████| 100000/100000 [00:00<00:00, 258963.28it/s]

for (ϵ=0.543656365691809, δ=0.006737946999085467) approximation, the loss is 0.1995


In [69]:
counter3 = CountMinSketch(3, 4)
counter3.evaluate()

100%|██████████| 100000/100000 [00:00<00:00, 370322.00it/s]

for (ϵ=0.6795704571147613, δ=0.049787068367863944) approximation, the loss is 0.19988


In [70]:
counter4 = CountMinSketch(2, 5)
counter4.evaluate()

100%|██████████| 100000/100000 [00:00<00:00, 494966.20it/s]

for (ϵ=0.543656365691809, δ=0.1353352832366127) approximation, the loss is 0.39906


In [71]:
counter5 = CountMinSketch(5, 3)
counter5.evaluate()

100%|██████████| 100000/100000 [00:00<00:00, 260432.63it/s]

for (ϵ=0.9060939428196817, δ=0.006737946999085467) approximation, the loss is 0.39831


In [72]:
counter6 = CountMinSketch(2, 3)
counter6.evaluate()

100%|██████████| 100000/100000 [00:00<00:00, 473862.51it/s]

for (ϵ=0.9060939428196817, δ=0.1353352832366127) approximation, the loss is 1.39934
